In [7]:
# 📁 03_model_training.ipynb
import os
import pandas as pd
import numpy as np
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

DATA_PATH = "../data/preprocessed_labeled.csv"
MODEL_PATH = "../model/sentiment_model.keras"
TOKENIZER_PATH = "../model/tokenizer.pkl"
LABEL_ENCODER_PATH = "../model/label_encoder.pkl"

assert os.path.exists(DATA_PATH), f"❌ 데이터 파일 없음: {DATA_PATH}"

df = pd.read_csv(DATA_PATH)
df.dropna(subset=["clean_title", "label"], inplace=True)

texts = df["clean_title"].astype(str).tolist()
labels = df["label"].tolist()

encoder = LabelEncoder()
y = encoder.fit_transform(labels)

with open(LABEL_ENCODER_PATH, "wb") as f:
    pickle.dump(encoder, f)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=30)

with open(TOKENIZER_PATH, "wb") as f:
    pickle.dump(tokenizer, f)

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=30),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

print("✅ 모델 학습 시작")
model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1,
    shuffle=True
)

model.save(MODEL_PATH)
print(f"✅ 모델 저장 완료: {MODEL_PATH}")
print("📊 전체 샘플 수:", len(df))
print("🧠 라벨 분포:", np.bincount(y))
print("📦 모델 저장:", MODEL_PATH)
print("📁 토크나이저 저장:", TOKENIZER_PATH)
print("📁 라벨인코더 저장:", LABEL_ENCODER_PATH)


c:\realestate_sentiment_project\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


✅ 모델 학습 시작
Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 916ms/step - accuracy: 0.3042 - loss: 1.0990 - val_accuracy: 0.4000 - val_loss: 1.0910
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - accuracy: 0.5437 - loss: 1.0849 - val_accuracy: 0.5000 - val_loss: 1.0835
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.6083 - loss: 1.0664 - val_accuracy: 0.5000 - val_loss: 1.0742
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - accuracy: 0.5479 - loss: 1.0429 - val_accuracy: 0.5000 - val_loss: 1.0645
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step - accuracy: 0.4958 - loss: 1.0297 - val_accuracy: 0.5000 - val_loss: 1.0551
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.5167 - loss: 0.9987 - val_accuracy: 0.4000 - val_loss: 1.0490
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 177ms/step - accuracy: 0.5000 - loss: 0.9816 - val_accuracy: 0.4000 - val_loss: 1.0547
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - accuracy: 0.4938 - loss: 0.9713 - val_accuracy: 0.4000 - va